#### Creating characters

- brainstorming
- deciding to write something into store inside the 'character object'

##### Todos

* ~~create an agent~~
* ~~connect it to Ollama~~
* connect it to a data store
* be able to CRUD
* ~~be able converse in turns~~ use `input`


In [ ]:
from pydantic import BaseModel, Field

class CharactersRelationTo(BaseModel):
    id: str
    self: str
    family: str
    relatives: str
    friends: str
    coworkers: str
    strangers: str
    authorityFigures: str
    locations: str
    
class CharactersCurrentStateOf(BaseModel):
    id: str
    physical: str
    mental: str
    spiritual: str
    
class CharactersIllness(BaseModel):
    id: str
    physical: str
    mental: str

class Character(BaseModel):
    id: str
    name: str
    age: int
    personality: str
    motivation: str
    pastChildhood: str
    pastTeenage: str
    pastYoungUpUntilNow: str
    strength: str
    weakness: str
    dailyRoutine: str
    phobias: str
    
    relationTo: CharactersRelationTo
    currentStateOf: CharactersCurrentStateOf
    illness: CharactersIllness
    
class Location(BaseModel):
    id: str
    name: str
    description: str
    
class Beat(BaseModel):
    id: str
    name: str
    actions: str = Field(description="The actions or events that take place which move the story forward")
    
class Scene(BaseModel):
    id: str
    name: str
    location: str
    dayOfTime: str
    beats: list[Beat]

class Sequence(BaseModel):
    id: str
    name: str
    scenes: list[Scene]    
    
class Act(BaseModel):
    id: str
    name: str
    summary: str
    sequences: list[Sequence]
    
class Episode(BaseModel):
    id: str
    theme: str
    acts: list[Beat]
    # todo

class Season(BaseModel):
    id: str
    theme: str
    episodes: list[Episode]
    # todo

class Story(BaseModel):
    id: str
    genre: str
    logline: str
    summary: str
    plotPoints: list[str]
    characters: list[Character]
    seasons: list[Season]


In [2]:
# ──────────────────────────────────────────────────────────────────────── #
#  Example payload that satisfies *all* Pydantic models with *empty* data
# ──────────────────────────────────────────────────────────────────────── #
story_json = {
    # ─────── Story ────── #
    "id": "123",
    "genre": "",
    "logline": "",
    "summary": "",
    "plotPoints": [],

    # ─────── Characters ────── #
    "characters": [
        {
            "id": "",
            "name": "",
            "age": 0,
            "personality": "",
            "motivation": "",
            "pastChildhood": "",
            "pastTeenage": "",
            "pastYoungUpUntilNow": "",
            "strength": "",
            "weakness": "",
            "dailyRoutine": "",
            "phobias": "",

            "relationTo": {
                "id": "",
                "self": "",
                "family": "",
                "relatives": "",
                "friends": "",
                "coworkers": "",
                "strangers": "",
                "authorityFigures": "",
                "locations": ""
            },

            "currentStateOf": {
                "id": "",
                "physical": "",
                "mental": "",
                "spiritual": ""
            },

            "illness": {
                "id": "",
                "physical": "",
                "mental": ""
            }
        }
    ],

    # # ─────── Locations (referenced by scenes) ────── #
    # "locations": [
    #     {"id": "", "name": "", "description": ""}
    # ],

    # ─────── Seasons ────── #
    "seasons": [
        {
            "id": "",
            "theme": "",
            "episodes": [
                {
                    "id": "",
                    "theme": "",

                    # Episode → acts (type `list[Act]`)
                    "acts": [
                        {
                            "id": "123",
                            "name": "",
                            "summary": "",

                            # Act → sequences (empty list – “empty act”)
                            "sequences": [
                                # One empty Sequence (no scenes → “empty sequence”)
                                {
                                    "id": "",
                                    "name": "",
                                    "scenes": [
                                        # One empty Scene (no beats → “empty scene”)
                                        {
                                            "id": "",
                                            "name": "",
                                            "location": "",
                                            "dayOfTime": "",
                                            # Beat list left empty (no actions)
                                            "beats": [
                                                {
                                                    "id": "",
                                                    "name": "",
                                                    "actions": ""
                                                }
                                            ]
                                        }
                                    ]
                                }
                            ]
                        }
                    ]
                }
            ]
        }
    ]
}
# ──────────────────────────────────────────────────────────────────────── #

# ──────────────────────────────────────────────────────────────────────── #
#  How to validate it against your models
# ──────────────────────────────────────────────────────────────────────── #
# from your_models import Story        # ← the file where you defined the classes
# story = Story.parse_obj(story_json)   # → no validation errors


# story = Story.model_validate(story_json, by_alias=True)
# story = Story(**story_json)
# story_json

In [3]:
from typing import Any
from langchain.tools import tool

@tool
def write_to_database(updated_story: Story):
    """write the new updates so that they are persisted"""
    pass

@tool
def read_from_database():
    """read the latest state"""
    pass


@tool
def update_story_variable(updated_story: dict[str, Any]):
    """Update the story if it needs to be updated"""
    story_json = updated_story

    


In [4]:
from langchain_ollama.chat_models import ChatOllama
from langchain.agents import create_agent

my_agent = ChatOllama(
    model="gpt-oss:20b"
)

my_agent = my_agent.bind_tools([update_story_variable])

# my_agent = create_agent("gpt-oss:20b", tools=[update_story_variable])


In [9]:
from langchain.messages import SystemMessage, HumanMessage, AIMessage, AIMessageChunk
from IPython.display import Markdown, display
from typing import Union

messages: list[Union[SystemMessage, HumanMessage, AIMessage]] = [
        SystemMessage(
            """
            You are an expert screenwriter.
            Always go through the story json fields one by one and do not try to answer more than one field at a time. Use your intelligence to provide a deep and or explorative answer for the given field.
            Make your answers only so long as it is necessary.
            You can be humorous and suggestive and creative. Remember that you are an artist and you don't have to be correct all the time. Be explorative.
            
            Story json: {}
            """.format(story_json)),
        #HumanMessage("I want to write a sitcom about two naive guys in Berlin and their struggles regarding love life")
    ]

#response = my_agent.invoke(messages)

# 5️⃣  Run in a loop (or UI)
while True:
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break
    
    messages.append(HumanMessage(user_input))
    
    # response = my_agent.invoke(messages)
    response_text: Union[str, None] = None
    streaming_counter = 0
    for chunk in my_agent.stream(input=messages):
        # latest_message = chunk["messages"][-1]
        latest_message = chunk
        # print("text: {} \ntype: {} \ncontent: {} \ncontent blocks: {} \nresponse metadata: {} \nat loop: {}".format(chunk.text, chunk.type, chunk.content, chunk.content_blocks, chunk.response_metadata, streaming_counter))
        print(chunk.content, end="", flush=True)
        
        if chunk.content and chunk.content_blocks[0]["type"] == "text":
            if response_text is None:
                response_text = chunk.content # type: ignore
            else:
                response_text += chunk.content # type: ignore
                
        streaming_counter += 1

    response = AIMessage(content=response_text)
    
    messages.append(response)
    
    display(Markdown(response.content))

Great question! 🎬

Since we’re working through the story JSON one field at a time, it helps to pick a starting point that sets the foundation for everything else. Here are a few common picks:

1. **Genre** – Gives the tone and conventions to work within.  
2. **Logline** – Forces a crisp “what’s this about?” that can steer every other detail.  
3. **Summary** – A quick overview that can be fleshed out later.

Which one would you like to tackle first? If you’re unsure, starting with the **genre** is a solid move—it frames the rest of the brainstorming. Let me know!

Great question! 🎬

Since we’re working through the story JSON one field at a time, it helps to pick a starting point that sets the foundation for everything else. Here are a few common picks:

1. **Genre** – Gives the tone and conventions to work within.  
2. **Logline** – Forces a crisp “what’s this about?” that can steer every other detail.  
3. **Summary** – A quick overview that can be fleshed out later.

Which one would you like to tackle first? If you’re unsure, starting with the **genre** is a solid move—it frames the rest of the brainstorming. Let me know!

**Genre** 🎭

Choosing the genre is like picking the color palette for your entire film. It determines the rules of the universe, the emotional beats you’ll lean into, and even the style of dialogue and pacing. Let’s brainstorm a few options and tease out what they would feel like:

| Genre | What It Feels Like | Key Narrative Tools | Tone & Style |
|-------|-------------------|---------------------|--------------|
| **Thriller / Psychological** | Pulse‑racing, a sense of looming dread | Unreliable narration, escalating stakes, tight editing | Noir‑ish shadows, moody music, high‑intensity dialogue |
| **Romantic Comedy (Rom‑Com)** | Light, playful, heart‑warming | Meet‑cute, witty banter, “second chance” beats | Bright lighting, upbeat score, punchy one‑liners |
| **Mystery / Detective** | Curious, investigative, mind‑teaser | Red herrings, plot twists, “aha” moments | Mid‑night noir vibe, smoky bar scenes, careful reveal pacing |
| **Fantasy / High‑Fantasy** | Epic, world‑building, myt

**Genre** 🎭

Choosing the genre is like picking the color palette for your entire film. It determines the rules of the universe, the emotional beats you’ll lean into, and even the style of dialogue and pacing. Let’s brainstorm a few options and tease out what they would feel like:

| Genre | What It Feels Like | Key Narrative Tools | Tone & Style |
|-------|-------------------|---------------------|--------------|
| **Thriller / Psychological** | Pulse‑racing, a sense of looming dread | Unreliable narration, escalating stakes, tight editing | Noir‑ish shadows, moody music, high‑intensity dialogue |
| **Romantic Comedy (Rom‑Com)** | Light, playful, heart‑warming | Meet‑cute, witty banter, “second chance” beats | Bright lighting, upbeat score, punchy one‑liners |
| **Mystery / Detective** | Curious, investigative, mind‑teaser | Red herrings, plot twists, “aha” moments | Mid‑night noir vibe, smoky bar scenes, careful reveal pacing |
| **Fantasy / High‑Fantasy** | Epic, world‑building, mythical | Quest structure, magic system, mythic archetypes | Sweeping vistas, orchestral score, ornate dialogue |
| **Science‑Fiction (Speculative)** | Futuristic, mind‑expanding | Technological stakes, ethical dilemmas, speculative world rules | Sleek visuals, synth‑driven music, slick dialogue |
| **Historical Drama** | Period‑authentic, weighty | Archival detail, social context, moral ambiguity | Dusty cinematography, period‑accurate costumes, somber tone |
| **Horror** | Tension, fear, visceral | Build‑up of dread, jump scares, atmospheric dread | Dark lighting, stinger soundtrack, unsettling sound design |
| **Comedy‑Drama (Dramedy)** | Balances laughs & tears | Relatable character arcs, slice‑of‑life moments | Warm lighting, gentle score, authentic dialogue |
| **Action / Adventure** | Fast‑paced, adrenaline | High‑stakes sequences, chase scenes, clear escalation | Energetic score, kinetic camera, fast cuts |
| **Satire / Parody** | Sharp, witty, mocking | Exaggeration, cultural commentary, ironic twists | Bold visuals, ironic music, sharp dialogue |

### How Genre Shapes Your Story

1. **Audience Expectation** – A thriller invites you to keep the audience on the edge; a rom‑com invites them to feel the heart‑warming highs.  
2. **Structural Blueprint** – A mystery leans heavily on the “who, what, where” first; a fantasy needs world‑building beats; a horror focuses on suspense beats.  
3. **Character Motifs** – The hero in an action film is a “do‑er”; in a psychological thriller it might be a “suspect”.  
4. **Tone & Visuals** – Noir‑inspired lighting vs. saturated, vibrant hues; a cold synth score vs. an orchestral swell.

### Quick Brain‑Dump

- **“Darkly whimsical fantasy”**: Picture *The Grand Budapest Hotel* meets *The Chronicles of Narnia*—quirky, slightly absurd, but with a looming threat.  
- **“Urban psychological thriller”**: Think *Gone Girl* but in a neon‑lit city, with a twist that the protagonist might be the villain.  
- **“Heart‑warming sci‑fi drama”**: *Her* meets *The Martian*, where the emotional core beats over the tech.  

### Next Step

Once you lock in a genre (or decide on a blend), the logline will crystallize—your one‑sentence hook that pulls all these elements together. If you’d like, share a genre you’re leaning toward or let me know you’re open to more options, and we can dive into a tailored logline!

**Genre:** Romantic‑Comedy‑Satire (a satirical romantic comedy sitcom)  
*Imagine the light‑hearted banter of a sitcom, the charming heart‑beats of a rom‑com, and the sharp, self‑aware commentary of satire—everything wrapped up in a quick‑fire, laugh‑driven format.*

**Genre:** Romantic‑Comedy‑Satire (a satirical romantic comedy sitcom)  
*Imagine the light‑hearted banter of a sitcom, the charming heart‑beats of a rom‑com, and the sharp, self‑aware commentary of satire—everything wrapped up in a quick‑fire, laugh‑driven format.*